In [2]:
import pandas as pd
import numpy as np

class CollegeAllotmentSystem:
    def __init__(self, students_path, preferences_path, seats_path):
        # Load data from CSV files
        self.students = pd.read_csv(students_path)
        self.preferences = pd.read_csv(preferences_path)
        self.seats = pd.read_csv(seats_path)
        
        # Initialize tracking of allotted seats
        self.allotment_results = []
        
    def process_allotment(self):
        # Sort students by rank (ascending order, rank 1 is best)
        sorted_students = self.students.sort_values('Rank')
        
        # Process each student in order of rank
        for _, student in sorted_students.iterrows():
            student_id = student['UniqueID']
            name = student['Name']
            gender = student['Gender']
            caste = student['Caste']
            rank = student['Rank']
            
            # Get student's preferences in order
            student_prefs = self.preferences[self.preferences['UniqueID'] == student_id]
            student_prefs = student_prefs.sort_values('PrefNumber')
            
            allotted = False
            college_id = 'N/A'
            institution = 'No College Available'
            pref_number = 'N/A'
            
            # Check each preference in order
            for _, pref in student_prefs.iterrows():
                college_id = pref['CollegeID']
                pref_number = pref['PrefNumber']
                
                # Check if seat is available in the student's caste
                college_seats = self.seats[self.seats['CollegeID'] == college_id]
                
                if not college_seats.empty:
                    college_seats = college_seats.iloc[0]
                    caste_column = caste  # Column name matches caste
                    
                    # Check if the caste exists in the seats dataframe and has available seats
                    if caste_column in college_seats and college_seats[caste_column] > 0:
                        # Allot the college to the student
                        self.seats.loc[self.seats['CollegeID'] == college_id, caste_column] -= 1
                        self.seats.loc[self.seats['CollegeID'] == college_id, 'TOTAL No. of students admitted'] += 1
                        
                        institution = college_seats['Institution']
                        allotted = True
                        break
            
            # Record the result in the specified format
            self.allotment_results.append({
                'UniqueID': student_id,
                'Name': name,
                'Gender': gender,
                'Caste': caste,
                'Rank': rank,
                'CollegeID': college_id if allotted else 'N/A',
                'Institution': institution,
                'PrefNumber': pref_number if allotted else 'N/A'
            })
        
        return pd.DataFrame(self.allotment_results)
    
    def analyze_results(self, results_df):
        total_students = len(results_df)
        allotted_students = len(results_df[results_df['CollegeID'] != 'N/A'])
        unallotted_students = total_students - allotted_students
        
        print(f"Total Students: {total_students}")
        print(f"Allotted Students: {allotted_students}")
        print(f"Unallotted Students: {unallotted_students}")
        print(f"Success Rate: {(allotted_students/total_students)*100:.2f}%")
        
        # Show caste-wise distribution
        caste_allotment = results_df.groupby('Caste')['CollegeID'].apply(
            lambda x: (x != 'N/A').sum()
        ).reset_index(name='Allotted')
        
        caste_total = results_df['Caste'].value_counts().reset_index()
        caste_total.columns = ['Caste', 'Total']
        
        caste_stats = pd.merge(caste_total, caste_allotment, on='Caste')
        caste_stats['Success Rate'] = (caste_stats['Allotted'] / caste_stats['Total']) * 100
        
        print("\nCaste-wise Allotment Statistics:")
        print(caste_stats.to_string(index=False))
        
        return allotted_students, unallotted_students
    
    def save_results(self, output_path):
        results_df = pd.DataFrame(self.allotment_results)
        results_df.to_csv(output_path, index=False)
        print(f"Results saved to {output_path}")
    
    def save_updated_seats(self, output_path):
        self.seats.to_csv(output_path, index=False)
        print(f"Updated seat availability saved to {output_path}")
    
    def generate_summary_report(self, output_path):
        results_df = pd.DataFrame(self.allotment_results)
        
        # Create summary statistics
        summary_data = {
            'Total Students': [len(results_df)],
            'Allotted Students': [len(results_df[results_df['CollegeID'] != 'N/A'])],
            'Unallotted Students': [len(results_df[results_df['CollegeID'] == 'N/A'])],
            'Success Rate': [len(results_df[results_df['CollegeID'] != 'N/A']) / len(results_df) * 100]
        }
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv(output_path, index=False)
        print(f"Summary report saved to {output_path}")

# Main execution
if __name__ == "__main__":
    # Initialize the allotment system with CSV file paths
    allotment_system = CollegeAllotmentSystem(
        'student_details.csv',
        'preference_data.csv',
        'institution_data.csv'
    )
    
    # Process the allotment
    results = allotment_system.process_allotment()
    
    # Display first 20 results
    print("College Allotment Results (First 20):")
    print(results.head(20).to_string(index=False))
    
    # Analyze results
    allotted, unallotted = allotment_system.analyze_results(results)
    
    # Display updated seat availability
    print("\nUpdated Seat Availability:")
    print(allotment_system.seats.to_string(index=False))
    
    # Save results to CSV
    allotment_system.save_results('allotment_results.csv')
    allotment_system.save_updated_seats('updated_seat_availability.csv')
    allotment_system.generate_summary_report('allotment_summary.csv')
    
    # Additional analysis for low matching rate
    if allotted < (allotted + unallotted) * 0.1:  # If less than 10% matching
        print("\n" + "="*50)
        print("LOW MATCHING RATE ANALYSIS")
        print("="*50)
        
        # Check if it's a seat availability issue
        # Assuming caste columns start from index 6 (adjust if needed)
        total_seats = allotment_system.seats.iloc[:, 6:].sum().sum()
        total_students = len(results)
        
        print(f"Total available seats: {total_seats}")
        print(f"Total students: {total_students}")
        
        if total_seats < total_students:
            print("Primary issue: Insufficient seats for all students")
        else:
            print("Primary issue: Mismatch between student preferences and available seats")
            
        # Check caste-wise seat availability vs demand
        caste_demand = results['Caste'].value_counts()
        caste_seats = allotment_system.seats.iloc[:, 6:].sum()  # Sum caste-specific seats
        
        print("\nCaste-wise demand vs supply:")
        for caste in caste_demand.index:
            demand = caste_demand[caste]
            supply = caste_seats.get(caste, 0)
            deficit = demand - supply
            print(f"{caste}: {demand} students, {supply} seats, {'Deficit' if deficit > 0 else 'Surplus'}: {abs(deficit)}")

College Allotment Results (First 20):
  UniqueID                  Name Gender Caste  Rank CollegeID             Institution PrefNumber
1115619330  CHOWDARI BHANUPRASAD   BOYS    SC     1       147              SHAMSHABAD          1
7433256962     SUNNAM ANJANEYULU   BOYS    BC     2       174                HUSNABAD          1
1076592646  PALLIKONDA KAVYA SRI  GIRLS    BC     3        34                METPALLE          1
4981653513     RYAGALLA RAVALIKA  GIRLS    BC     4        36            CHINTHAKUNTA          1
9963405323   KUMMARIKUNTLA RAMYA  GIRLS    SC     5       345                   ALAIR          1
5196578828       PIRANGI SRAVANI  GIRLS    SC     6       187                   MEDAK          1
7236943884    MASAPAKA JAHANGEER   BOYS    SC     7       169                 CHERIAL          1
4568514578     SHETTI GNANESHWAR   BOYS    BC     8       203                 SINGOOR          6
5294129176         BOTTU NIROOPA  GIRLS    SC     9       344                   ALAIR    